[View on GitHub](https://github.com/csc-training/sscc-cp2k)

# Proton transfer simulations with CP2K

## Part 3: Re-converging the NEB minimum energy profile using the PBE0 hybrid functional

Due to self-interaction error and resulting electron overdelocalization, reaction barriers are typically underestimated by semilocal exchange–correlation functionals such as PBE. Hybrid functionals that include a fraction of exact Hartree–Fock (HF) exchange can mitigate this. We will now re-converge the energies of the NEB images optimized at the PBE level using the **PBE0 hybrid functional**.

### Step 3.1: Prepare the input file and submit the calculation

**Start by copying the wavefunction restart and coordinate file of each image to the current working directory:**

In [ ]:
%%bash
cp ../Part-1-NEB/*.wfn .
cp ../Part-1-NEB/*Replica* .

CP2K has a built-in `FARMING` program that allows running multiple inputs at the same time within a single job. To re-converge each NEB image, the farming input would look like as follows. **Run the cell to write the input to a file `farming.inp`.**

In [ ]:
%%writefile farming.inp
&GLOBAL
  PROJECT farming                         ! Project name, gets prepended to all output files
  PROGRAM FARMING                         ! Define farming-type job
  RUN_TYPE NONE                           ! Farming master input should have RUN_TYPE NONE
&END GLOBAL
&FARMING
  NGROUPS 8                               ! How many subjobs to run simultaneously
  &JOB                                    ! Input files for each subjob follows here
    INPUT_FILE_NAME hybrid-1.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-2.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-3.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-4.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-5.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-6.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-7.inp
  &END JOB
  &JOB
    INPUT_FILE_NAME hybrid-8.inp
  &END JOB
&END FARMING

As you can see, we need a separate input file for each subjob, in this case one for each image, `hybrid-1.inp` through `hybrid-8.inp`. **Inspect the input file `hybrid-8.inp` and fix all occurrences of `FIXME` based on that:**

* We just calculate the energy, so run type should be `ENERGY`
* Wavefunction restart file name is `enol-neb-BAND8-RESTART.wfn`
* Coordinate file name is `enol-neb-pos-Replica_nr_8-1.xyz`
* PBE0 functional *replaces* 25% of the PBE exchange with exact Hartree–Fock (HF) exchange

Rest of the inputs are provided and do not need to be edited.

In [ ]:
%%writefile hybrid-8.inp
&GLOBAL
  PROJECT enol-hybrid-8                   ! Project name, gets prepended to all output files
  RUN_TYPE FIXME                          ! Type of calculation
  PRINT_LEVEL LOW                         ! Low verbosity
&END GLOBAL

&FORCE_EVAL                               ! Include DFT and system settings
  &DFT
    RESTART_FILE_NAME FIXME
    BASIS_SET_FILE_NAME BASIS_MOLOPT
    &QS
      EPS_DEFAULT 1.0E-12                 ! Sets convergence criteria so that energy should be accurate up to this value
    &END QS
    &POISSON                              ! How to deal with electrostatics
      PERIODIC NONE                       ! System is not periodic
      POISSON_SOLVER WAVELET              ! Efficient solver for non-periodic system
    &END POISSON
    &SCF
      SCF_GUESS RESTART                   ! How to guess initial electron density for SCF cycle
      EPS_SCF 1.0E-6                      ! Energy convergence criteria (atomic units)
      &OT                                 ! Use orbital transformation method instead of brute-force diagonalization
        ALGORITHM IRAC                    ! OT algorithm
        MINIMIZER DIIS                    ! Minimization algorithm
        PRECONDITIONER FULL_ALL           ! Preconditioner for the minimizer
      &END OT
      MAX_SCF 20                          ! Maximum number of SCF steps before rebuilding preconditioner
      &OUTER_SCF ON
        EPS_SCF 1.0E-6                    ! Outer loop energy convergence criteria (atomic units)
        MAX_SCF 12                        ! Maximum number of preconditioner rebuilds before terminating
      &END OUTER_SCF
      &PRINT
        &RESTART
          BACKUP_COPIES 0                 ! Avoid excessive printing of restart files
        &END RESTART
      &END PRINT
    &END SCF
    &XC
      &XC_FUNCTIONAL
        &PBE
          SCALE_X FIXME                   ! Include 75% PBE exchange
          SCALE_C 1.0                     ! Include 100% PBE correlation
        &END PBE
      &END XC_FUNCTIONAL
      &HF
        FRACTION FIXME                    ! Include 25% HF exchange
      &END HF
    &END XC
  &END DFT
  &SUBSYS
    &CELL
      PERIODIC NONE                       ! System is not periodic
      ABC 15.0 15.0 15.0                  ! Dimensions of the simulation cell in angstroms
    &END CELL
    &TOPOLOGY                             ! Specify input coordinate format and filename
      COORD_FILE_FORMAT XYZ
      COORD_FILE_NAME FIXME
    &END TOPOLOGY
    &KIND H                               ! Basis sets and pseudopotential for hydrogen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q1
    &END KIND
    &KIND C                               ! Basis sets and pseudopotential for carbon
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q4
    &END KIND
    &KIND O                               ! Basis sets and pseudopotential for oxygen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q6
    &END KIND
  &END SUBSYS
&END FORCE_EVAL

**Next, write the following simple batch job script to a file by running the cell.** We'll again request two full Puhti nodes (40 CPU cores each), meaning that each of the 8 images will be run using $80/8=10$ cores.

In [ ]:
%%writefile job.sh
#!/bin/bash
#SBATCH --time=00:15:00
#SBATCH --reservation=sscc_thu_large
#SBATCH --partition=large
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=40
#SBATCH --account=project_2006657

module purge
module load intel-oneapi-compilers-classic/2021.6.0
module load intel-oneapi-mpi/2021.6.0
module load cp2k/2024.1

srun cp2k.psmp farming.inp

**Run the calculation as a batch job by executing the cell below.** Note, before running `sbatch`, we unset all Slurm environment variables as we are submitting a batch job from another Slurm job (this notebook that we're currently running).

In [ ]:
!unset ${!SLURM@}; sbatch job.sh

The calculation should only take about 2 minutes. You may monitor the queue using `squeue` command:

In [ ]:
!squeue --me

### Step 3.2: Analysis

**Once the job has completed, extract the PBE0 energies into a file by running the cell below:**

In [ ]:
!grep "ENERGY|" enol-hybrid* | awk '{print $10}' > energy-pbe0.txt

**Then, plot the PBE0 energy profile together with the PBE profile (in meV):**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Read the energy of each image, convert to meV and shift IS energy to 0
energy_pbe = np.loadtxt('../Part-1-NEB/energy-pbe.txt')
energy_pbe = 27211.38*(energy_pbe-energy_pbe[0])
energy_pbe0 = np.loadtxt('energy-pbe0.txt')
energy_pbe0 = 27211.38*(energy_pbe0-energy_pbe0[0])

plt.plot(energy_pbe, 'o-', label='PBE')
plt.plot(energy_pbe0, 'o-', label='PBE0')
plt.xlabel('NEB image')
plt.ylabel('Energy (meV)')
plt.legend()
plt.tight_layout()
plt.show()

* Did the barrier height increase compared to the PBE result?

**Calculate the equilibrium constant for the proton transfer based on the PBE0 result and:**

$$K\approx\exp\left(-\Delta E/k_\mathrm{B}T\right)$$

**where $\Delta E=E_\mathrm{FS}-E_\mathrm{IS}$ is the reaction energy.**

In [ ]:
kT = 25.7  # Boltzmann's constant times 298 K in units of meV
K = np.exp(-energy_pbe0[-1]/kT)
print(K)

NMR results suggest that there should be a 76%–24% split between the IS and FS enol forms, respectively. **Estimate this split based on the calculated equilibrium constant.** Assume:

$$[\mathrm{IS}]+[\mathrm{FS}]=1$$
$$\frac{[\mathrm{FS}]}{[\mathrm{IS}]}=K$$

In [ ]:
# Solve the above system of equations for [IS] and [FS]!

* How well does your result agree with the experimental observation?

**[Finish the hands-on with Part 4.](../Part-4-AIMD/Part-4-AIMD.ipynb)**